In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
from matplotlib import pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from keras.models import Sequential
from keras.layers import Dense,Activation
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
import seaborn as sns
from statistics import mode
# Any results you write to the current directory are saved as output.

In [ ]:
class NeuralNetKeras:
    def load_data(self,train_data,test_data):
        self.train=pd.read_csv(train_data)
        self.test=pd.read_csv(test_data)
        pass
    def merge_all(self):
        all_feat=list(self.train.columns)
        self.train['Source_data']='train'
        self.test['price_doc']=0.0
        self.test['Source_data']='test'
        self.merged_data=self.train.append(self.test)
        pass
    
    def summary(self):
        self.summary_info={}
        for col in self.train:
            self.summary_info[col]={'Missing':len(self.train[pd.isnull(self.train[col])]),'Min':np.min(self.train[col]),'Max':np.max(self.train[col]),'N_Unique':len(pd.unique(self.train[col]))}
        pass
    def boxplots(self,vars_list):
        for i in range(len(vars_list)):
            plt.figure(i)
            sns.boxplot(self.train[vars_list[i]])
    def HotEncoding(self,col_list):
        for col in col_list:
            if self.merged_data[col].dtype=='object':
                self.merged_data.loc[:,col]=pd.Categorical.from_array(self.merged_data.loc[:,col]).labels
            if self.summary_info[col]['N_Unique']<=20:
                hot_encode=OneHotEncoder()
                hot_encode.fit(np.array(self.merged_data.loc[:,col]))
                self.merged_data.loc[:,col]=hot_encode.transform(np.array(self.merged_data.loc[:,col]))
        pass
    def setFullSq(self,data):
        missing_full=(data['full_sq']==0)|(pd.isnull(data['full_sq']))
        data.loc[missing_full,'full_sq']=data.loc[missing_full,'life_sq']
        return data
    def setSq(self,data,col,vs_col,imp_criteria):
        trouble=(data[col]==0)|(pd.isnull(data[col]))|(data[col]>data[vs_col])
        data.loc[trouble,col]=data.loc[trouble,vs_col]*self.all_area_summary[imp_criteria]['50%']
        return data
    def handle_sq_ft(self):
        area_vars=['full_sq','life_sq','kitch_sq']
        tr_te_merge=self.merged_data[area_vars]
        all_sq_ft=(pd.notnull(tr_te_merge['full_sq']))&(pd.notnull(tr_te_merge['life_sq']))&(pd.notnull(tr_te_merge['kitch_sq']))&(tr_te_merge['full_sq']!=0)&(tr_te_merge['life_sq']!=0)&(tr_te_merge['kitch_sq']!=0)
        tr_te_merge=tr_te_merge.loc[all_sq_ft,:]
        #remove outliers from each of the sq_ft columns
        no_outlier=(tr_te_merge['full_sq']<=np.mean(tr_te_merge['full_sq'])+3*np.std(tr_te_merge['full_sq']))&(tr_te_merge['full_sq']>=np.mean(tr_te_merge['full_sq'])-3*np.std(tr_te_merge['full_sq']))&(tr_te_merge['life_sq']<=np.mean(tr_te_merge['life_sq'])+3*np.std(tr_te_merge['life_sq']))&(tr_te_merge['life_sq']>=np.mean(tr_te_merge['life_sq'])-3*np.std(tr_te_merge['life_sq']))&(tr_te_merge['kitch_sq']<=np.mean(tr_te_merge['kitch_sq'])+3*np.std(tr_te_merge['kitch_sq']))&(tr_te_merge['kitch_sq']>=np.mean(tr_te_merge['kitch_sq'])-3*np.std(tr_te_merge['kitch_sq']))
        tr_te_merge=tr_te_merge.loc[no_outlier,:]
        # Generate variable %Kitch/Full, %Kitch/Life and %Life/Full
        tr_te_merge.loc[:,'%Life/Full']=tr_te_merge.loc[:,'life_sq']/tr_te_merge.loc[:,'full_sq']
        tr_te_merge.loc[:,'%Kitch/Full']=tr_te_merge.loc[:,'kitch_sq']/tr_te_merge.loc[:,'full_sq']
        tr_te_merge.loc[:,'%Kitch/Life']=tr_te_merge.loc[:,'kitch_sq']/tr_te_merge.loc[:,'life_sq']
        self.all_area_summary=tr_te_merge.describe()
        # Impute values for missing sq ft
        #Full Sq
        self.merged_data=self.setFullSq(self.merged_data)
        #Kitch Sq
        self.merged_data=self.setSq(self.merged_data,'kitch_sq','full_sq','%Kitch/Full')
        #Life Sq
        self.merged_data=self.setSq(self.merged_data,'life_sq','full_sq','%Life/Full')
        pass
    def impute_mode(self,col_list):        
        for col in col_list:
            all_data=self.merged_data[col]
            all_data=all_data[pd.notnull(all_data)]
            mode_value=mode(all_data)
            missing=len(self.merged_data[pd.isnull(self.merged_data[col])])
            self.merged_data.loc[pd.isnull(self.merged_data[col]),col]=mode_value
            print('Missing ',col,'=',missing,' out of ',len(self.merged_data[col]) ,'replaced by mode value= ',mode_value)
            pass
    def handle_missing(self,impute_cols):
        # Handles missing sq ft data and imputes data and removes anomaly in the data
        self.handle_sq_ft()
        self.impute_mode(impute_cols)
        #self.handle_missing_categories()
        #self.handle_missing_continuous()
        pass   
    def preprocess_data(self):
        self.HotEncoding()
        pass
    def house_features(self):
        self.house_feat=list(self.merged_data.iloc[:,2:11].columns)
        print (self.house_feat)
    def correlation_mat(self):
        self.corr_mat=self.train.corr()
        pass
    def no_missing_no_string(self):
        self.no_missing_cols=[]
        for key in self.summary_info:
            if self.summary_info[key]['Missing']==0 and key not in ['timestamp','id','price_doc'] and self.train[key].dtype not in ['object']:
                self.no_missing_cols.append(key)
        pass
    def descriptive_stats(self):
        self.desc_stats=self.train.describe()
        pass
    def build_model_skeleton(self,n_nodes,activation_fn,i_dim):
        self.model=Sequential()
        self.model.add(Dense(n_nodes,input_dim=i_dim))
        self.model.add(Activation(activation_fn))
        self.model.add(Dense(1, kernel_initializer='normal'))  
        pass
    def compile_model(self,optimizer_crit,loss_crit):
        self.model.compile(optimizer=optimizer_crit,loss=loss_crit)
        pass
    def fit_model(self,n_epochs,b_size,data_train,data_test):
        self.model.fit(x=np.array(data_train[self.house_feat]),y=np.array(data_train['price_doc']),epochs=n_epochs,batch_size=b_size)
    def predict_model(self,b_size,data_train,data_test):
        self.predictions=self.model.predict(x=np.array(data_test),batch_size=b_size)
    pass

In [ ]:
n1=NeuralNetKeras()
n1.load_data('../input/train.csv','../input/test.csv')
n1.summary()
n1.merge_all()
n1.house_features()
n1.HotEncoding(n1.house_feat)
n1.handle_missing(impute_cols=['floor','max_floor','material','build_year','num_room','state','product_type'])

In [ ]:
n1.build_model_skeleton(10,'relu',len(n1.house_feat))
n1.compile_model('rmsprop','mse')
n1.train=n1.merged_data.loc[n1.merged_data['Source_data']=='train',n1.house_feat+['price_doc']]
n1.test=n1.merged_data.loc[n1.merged_data['Source_data']=='test',n1.house_feat]                        
#n1.fit_model(20000,10000,n1.train,n1.test)
#n1.predict_model(10000,n1.train,n1.test)
print(n1.train.head())
print(n1.test.head())

In [ ]:
#final=pd.DataFrame(n1.merged_data.loc[n1.merged_data['Source_data']=='test','id'])
#final['prediction']=n1.predictions
#final.head()

In [ ]:
#final.to_csv('final.csv')